# Equivariance Tutorial

In [ ]:
import torch
from torch.nn.functional import affine_grid, grid_sample, pad
from torchvision.io import read_image
import lietorch
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
extent = (-1, 1, -1, 1)

In [ ]:
plt.rcParams.keys()

In [ ]:
shoe = read_image("images/shoe.png")[None, ...] / 255.
dress = read_image("images/dress.png")[None, ...] / 255.

In [ ]:
def transform(image, x, A, padding=True):
    """Apply action (x, A) to image with circular padding."""
    B, C, H, W = image.shape
    x = torch.tensor([-1, 1]) * x
    affine_matrix = torch.hstack((A, x[None, ...].T / 3))
    if padding:
        image = pad(image, (W, W, H, H), mode="circular")
        grid = affine_grid(affine_matrix[None, ...], (B, C, 3 * H, 3 * W), align_corners=False)
        return grid_sample(image, grid, align_corners=False)[..., H:2*H, W:2*W]
    else:
        grid = affine_grid(affine_matrix[None, ...], (B, C, H, W), align_corners=False)
        return grid_sample(image, grid, align_corners=False)

## Theory

### Lie Groups

> **Definition (Lie Group)** $G$ is a _Lie group_ if it is 
> 1. a _smooth manifold_ - so smooth and looks locally like $\mathbb{R}^n$ - and
> 2. a _group_ - we have a smooth, well-behaved product $\cdot: G \times G \to G$.

The most important Lie groups (imo) encode symmetries on other spaces. 

> **Example (Translation Group)**
> The $n$-dimensional _translation group_ $\mathbb{R}^n$ acts on Euclidean space $\mathbb{R}^n$ by translation, namely
> $$ (\mathbf{x}, \mathbf{y}) \mapsto \mathbf{x} + \mathbf{y}, $$
> and has group product
> $$ (\mathbf{x}, \mathbf{y}) \mapsto \mathbf{x} + \mathbf{y}. $$
Of course, this is an incredibly boring example. 
Slightly less trivial is the following:
> **Example (Special Orthogonal Group)**
> The _special orthogonal group_ $\operatorname{SO}(n)$ acts on Euclidean space $\mathbb{R}^n$ by rotation, namely
> $$ (R, \mathbf{y}) \mapsto R\mathbf{y}, $$
> and has group product
> $$ (R, S) \mapsto RS. $$
Here, we represent the elements of $\operatorname{SO}(n)$ as $n \times n$ orthogonal matrices with determinant $1$. 
For example, the counter-clockwise rotation by angle $\theta$ is given by
$$ R = \begin{pmatrix} \cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta) \end{pmatrix}. $$


Note that in this example the group and the space that is acted on can no longer be identified.

We get our favourite group by combining the two previous ones:
> **Example (Special Euclidean Group)**
> The _special Euclidean group_ $\operatorname{SE}(n)$ acts on Euclidean space $\mathbb{R}^n$ by roto-translation, namely
> $$ ((\mathbf{x}, R), \mathbf{y}) \mapsto \mathbf{x} + R\mathbf{y}, $$
> and has group product
> $$ ((\mathbf{x}, R), (\mathbf{y}, S)) \mapsto (\mathbf{x} + R\mathbf{y}, RS). $$

Many problems have inherent symmetries. For example, if we want to classify the object in an image, rotating the object shouldn't change the classification: 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].set_title("Shoe")
ax[1].set_title("Still a shoe")
ax[0].imshow(shoe.squeeze())
x = torch.tensor([0., 0.])
theta = torch.tensor([-0.862364]) # Random number
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])
ax[1].imshow(transform(shoe, x, R, padding=False).squeeze());

Lie groups give us a mathematically formal way of thinking about these symmetries.

### Group Representations
An important ingredient in this mathematical formalism is the notion of _group representations_:
> **Definition (Group Representation)** Let $G$ be a Lie group and $V$ a vector space. Then we call a homomorphism $\mathcal{U} : G \to \operatorname{GL}(V)$ a _group representation_.

The idea is that we can represent elements of $G$ as operators on $V$.
(Quasi-)regular representations form the most important class of representations for us.
> **Definition (Quasi-Regular Representation)** Let $G$ act transitively on $X$, i.e. for all $x, y \in X$ there exists a $g \in G$ such that $g x = y$.
> Then the induced _quasi-regular representation_ $\mathcal{U}: G \to \operatorname{GL}(\mathbb{L}_2(X))$ is given by
> $$ \mathcal{U}_g f(x) := f(g^{-1} x). $$
In fact, we already saw such a quasi-regular representation: we rotated the shoe above (say in $\mathbb{L}_2(\mathbb{R}^2)$) by rotating the domain (say $\mathbb{R}^2$) the other way.
$G$ naturally acts on itself; in this case the induced quasi-regular representation is simply called the _regular representation_.

### Equivariance

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
x = torch.tensor([0., 0.])
R = torch.tensor([[1., 0.], [0., 1.]])
ax[0].imshow(transform(shoe, x, R).squeeze(), extent=extent)

# Your roto-translation here ⬇️
x = torch.tensor([0., 0.])
R = torch.tensor([[1., 0.], [0., 1.]])
ax[1].imshow(transform(shoe, x, R).squeeze(), extent=extent);

### Lifting

## Application